### Realizando todos os imports que serão necessário para o equacionamento do TG

In [60]:
import sympy  as sp
import numpy as np 
import pandas as pd 
import math as mt 
import matplotlib as plt

In [223]:
x, y, z, phi, T, T_max, T1, T2, T4,T_mr,delta_T_phi,delta_T_zp,delta_T_zs, delta,P,  P1, P2, P3, A_ref, A_h_eff, phi_ZP, theta, q_ref, A_h_reff, delta_P_3_4, R, m_ponto_3, T3, k, D_ref, phi_global, m_ponto_zp, phi_pobre, phi_rico,A_ft, D_ref, L_cc, L_zp, L_zs, L_zd,L_sz, phi_Zs, m_ponto_arref, m_ponto_zd, eta_zr, eta_zp, eta_zs, p_3,T_med_zr, T_max_zr, delta_p, psi_t3, m_ponto_comb,V_zs   = sp.symbols(
    [
    'x','y','z', 'phi', 'T', 'T_max', 'T1', 'T2', 'T4', 'T_mr', 'delta_T_phi', 'delta_T_zp','delta_T_zs', 'delta','P', 'P1',
    'P2', 'P3', 'A_ref', 'A_h_eff', 'phi_ZP', 'theta', 'q_ref', 'A_h_reff', 
    'delta_P_3_4','R', 'm_ponto_3','T3', 'k', 'D_ref', 'phi_global', 'm_ponto_zp', 'phi_pobre', 'phi_rico','A_ft','D_ref','L_cc','L_zp','L_zs','L_zd', 'L_sz','phi_Zs','m_ponto_arref', 'm_ponto_zd','eta_zr','eta_zp','eta_zs', 'p_3', 'T_med_zr', 'T_max_zr','delta_p', 'psi_t3', 'm_ponto_comb', 'V_zs'
    ])



### Funcao da equação (5):

𝐶𝑥𝐻𝑦 + 𝑎𝑂2 → 𝑥𝐶𝑂2 + (𝑦⁄2)𝐻2O

In [224]:
 
def razao_ar_comb_esteq(a, PMar, PMcomb):
    A_C_esteq = a*(1 + 3.7274 + 0.0458) * (PMar/PMcomb)
    return A_C_esteq

teste = razao_ar_comb_esteq(17.75, 28.9645, 167.3087)
# phi_esteq = 1/teste
# phi_esteq


### razão de equivalência
A razão de equivalência é uma relação frequentemente usada para indicar se a mistura
está pobre, rica ou está estequiométrica. A razão de equivalência é definida como:

𝜙 =
(𝐴/𝐶)𝑒𝑠𝑡𝑒𝑞
(𝐴/𝐶)𝑟𝑒𝑎𝑙
=
(𝐶/𝐴)𝑟𝑒𝑎𝑙
(𝐶/𝐴)𝑒𝑠𝑡𝑒q

In [225]:
def razao_equivalencia(A_C_esteq, A_C_real):
    phi = A_C_esteq/A_C_real
    return ('rica' if phi > 1 else 'pobre')




DUVIDA : COMO FAREMOS COM RELAÇÃO AO EQUACIONAMENTO DA Temperatura de Chama Adiabática ? 

### Turbina a gás

- Componente 1:
    - Compressor, que recebe potência de eixo da turbina, converte em energia que é
entregue ao ar elevando a sua entalpia e, por consequência, a pressão e a temperatura
desse fluido de trabalho

- Componente 2:
    - Câmara de combustão, onde o ar, proveniente do compressor, é misturado ao
combustível, formando os reagentes. É fornecida energia de ativação para que a
combustão desses reagentes inicie e se processe em seu interior


- Componente 3:
    - Turbina, dispositivo mecânico ideal que converte a entalpia dos gases quentes advindos
da câmara de combustão em energia mecânica em potência de eixo

### Perda de pressao quente/fria

- Sendo 𝑃1 e 𝑃2 as pressões totais nas seções transversais logo ao início e logo ao final do difusor, respectivamente

- Onde os parâmetros 𝐴𝑟𝑒𝑓 e 𝑞𝑟𝑒𝑓 são, respectivamente, a área transversal e a pressão
dinâmica avaliadas num plano de máxima área transversal da câmara de combustão, delimitada
pelo seu invólucro

- Já o termo 𝐴ℎ,𝑒𝑓𝑓, que representa a área efetiva total dos orifícios do tubo
de chama, pode ser calculado de duas formas

In [226]:
def perda_pressao_quente(rho, u, T4, T3):
    delta_P_quente = 0.5 * rho * (u**2) * ((T4/T3) - 1 )
    return delta_P_quente



delta_P_difusor = P1 - P2  #Sendo 𝑃1 e 𝑃2 as pressões totais nas seções transversais logo ao início e logo ao final do difusor, respectivamente

def P_turboChama(A_ref, A_h_eff, q_ref):   

    delta_P_turbo_chama = ((A_ref/A_h_eff) ** 2) * q_ref

    return delta_P_turbo_chama

def perda_pressao_fria(P_difusor, P_turbo_chama):
    delta_P_fria = sum(P_difusor, P_turbo_chama)
    return delta_P_fria


### Dimensionalizando os orificios do tubo de chama:

In [227]:
def dimensao_orificio_tubo_chama(A_ref, delta_p_3_4, q_ref, delta_P_difusor):
    dimensao_orificioTuboChama = A_ref/(((delta_p_3_4/q_ref) - (delta_P_difusor/q_ref))**0.5)
    return dimensao_orificioTuboChama

### Calculando A_h_reff

In [228]:
#Como esperando muitos valores de entrada para fazermos o somatório, passaremos como entrada uma lista

C_d = []
A_h = []

def A_h_eff_function(C_d, A_h):
    somatorio_A_h_reff = 0
    for c in range(len(C_d)):
        somatorio_A_h_reff =  somatorio_A_h_reff + C_d[c] * A_h[c]  
        
    return somatorio_A_h_reff

# A_h_eff_function(C_d, A_h)


### Parametros para especificar a perda de pressão

- O primeiro (∆𝑃3−4/𝑃3), chamado de perda de pressão global, geralmente assume
valores entre 4% a 8%, dependendo do tipo de câmara de combustão. O segundo termo
(∆𝑃3−4/𝑞𝑟𝑒𝑓), chamado de fator de perda de pressão, é mais expressivo para o projetista pois
denota a resistência do fluido em atravessar a câmara de combustão. Aerodinamicamente, pode
ser considerado um equivalente ao “coeficiente de arrasto”. Ambos os termos não consideram
a perda “quente” 

In [229]:
delta_P_3_4 = P3 * (delta_P_3_4/q_ref) * (R/2) * ( (((m_ponto_3 * T3)**0.5)/ (A_ref*P3))**2 )

delta_P_3_4

R*delta_P_3_4*(T3*m_ponto_3)**1.0/(2*A_ref**2*P3*q_ref)

### Qualidade Transversal de temp e Perfil Radial de temp


- São dois os termos usados para descrever o perfil de temperatura transversal: a
Qualidade Transversal de Temperatura (em inglês, Temperature Factor, Traverse Number ou
Pattern Factor) e o Perfil Radial de Temperatura (em inglês, Profile Factor), expressos
matematicamente a seguir:

In [230]:
def qualidade_tranversal_temp(T_max, T4, T3):
    qualidade_trans_temp = (T_max - T4)/(T4 - T3)
    return qualidade_trans_temp

def perfil_radial_temp(T_mr, T4, T3):
    perfil_rad_temp = (T_mr - T4)/(T4 - T3)
    return perfil_rad_temp

#Onde 𝑇𝑚𝑎𝑥 é a máxima temperatura dos gases na saída da câmara de combustão e 𝑇𝑚𝑟 é a temperatura média mais alta de uma linha de circunferência escolhida
    

### Informacões para dimensionar do combustor

- Vazão mássica
- Vazão mássica de combustível
- Distribuição de temperatura, pressão e velocidade na entrada do combustor;
- Perfil transversal de temperatura na saída do combustor;
- Limitações de perda de pressão;
- Tipos de poluentes e níveis permitidos
- Temperatura na parede permitida
- Limitações de tamanho
- Limitações de peso
- Requisitos de durabilidade e semelhança de vida útil entre as trocas de peças;
- Compatibilidade com o tipo de combustível;
- Outras limitações ou requisitos especiais.


### Calculo da área de transfência

- A constante 𝑘 varia apenas com relação às unidades usadas, é igual à 143,5 ao utilizar
a unidades do sistema internacional (SI) e é igual à 0,83, nas unidades 𝑙𝑏, 𝑓𝑡 𝑒 𝑅. O termo 𝑚̇3
é vazão mássica de ar, 𝑃3 é a pressão total e 𝑇3 a temperatura estimadas na entrada da câmara,
𝑞𝑟𝑒𝑓 é a pressão dinâmica na máxima área da seção transversal da carcaça (𝐴𝑟𝑒𝑓) e ∆𝑃3−4 é a
perda de pressão total ao longo da câmara.
Já  é a variável que se relaciona com a eficiência da combustão, . Como se busca
obter a maior eficiência possível, é utilizado o parâmetro  correspondente, tendo seu valor
igual à 73 . 106
(unidades do SI)

- O fator de correção da temperatura, b, é definido de acordo com a relação ar combustível
da zona primária. Para razão de equivalência nesta região entre 0,6 e 1,0, a fórmula 𝑏 =
245 (1,39 + ln 
𝑍𝑃) é empregada. Já para 
𝑍𝑃 entre 1,0 e 1,4

- A razão 𝑚̇𝑍𝑃 𝑚̇ ⁄ 3
, ou melhor, a porcentagem de ar a ser designada à zona primária, deve
ser escolhida de modo que a razão de equivalência no local esteja na região de flamabilidade
da mistura

In [231]:

def area_tranferencia_aerodinamica(k, m_ponto_3m, T3, P3, delta_P_3_4, q_ref):
    area_transf = k * ( ((m_ponto_3 * (T3**0.5) )/(P3))**2  * ((delta_P_3_4/q_ref)/(delta_P_3_4/P3)) ** 0.5 )
    return area_transf

def area_tranferencia_quimica(P3, A_ref, D_ref, T3, b, m_ponto_3):
    theta = ((P3**1.75) * A_ref * (D_ref ** 0.75) * np.exp(T3/b))/ m_ponto_3
    return theta



In [232]:

def phi_global_function(m_comb_op, m_ponto_3_op, m_comb, m_ponto_3):
    phi_global = (m_comb_op/m_ponto_3_op)/ (m_comb/m_ponto_3)

def phi_zp_function(phi_global, m_ponto_zp, m_ponto_3):
    phi_ZP = phi_global/(m_ponto_zp/m_ponto_3)
    return phi_ZP

### Definindo o phi pobre e o phi rico

In [233]:
def phi_pobre_function(T3):
    phi_pobre = 0.70547 - 0.00046 * T3
    return phi_pobre
def phi_rico_function(T3):
    phi_rico = 1.46695 + 0.00172 * T3
    return phi_rico

### Determinação da Área do Tubo de Chama pagina 54 eq 34

In [234]:
def area_tubo_de_chama(A_ref):
    A_ft = 0.56*A_ref
    return A_ft

Altura de Referência e Altura do Tubo de Chama
A expressão para o cálculo da altura de referência (𝐷𝑟𝑒𝑓) e da altura do tubo de chama (𝐷𝑓𝑡) é diferente a depender do tipo de combustor o qual foi escolhido para o projeto. Baseandose na expressão relativa a área de um círculo de raio 𝑟 (área=𝜋𝑟2), pode-se chegar nas expressões contidas na Tabela 6, a partir das relações geométricas de cada tipo de combustor, como ilustrado a seguir.

Altura de referencia (D_ref)

In [235]:
def altura_referencia(A_ref,D_int):
    D_ref = (np.sqrt([(4*A_ref)/np.pi()]-pow(D_int,2))-D_int)/2
    return D_ref

Altura do Tubo de Chama (D_ft)

In [236]:
def altura_tubo_chama(A_ft,D_int,D_ref):
    D_ft = (A_ft/(np.pi()*(D_int+D_ref)))
    return D_ft

Determinação do Comprimento das Zonas (36)
Com base os estudos de Melconian e Modak , o comprimento da zona primária (𝐿𝑍𝑃) pode ser estimado como estando entre 2/3 e 3/4 da altura do tubo de chama – caso a limitação de espaço permita, a escolha último valor é preferível já que concede uma maior eficiência de combustão. Como parte da zona primária, a zona de recirculação tem seu comprimento sendo aproximadamente metade da altura do tubo de chama. O comprimento da zona secundária (𝐿𝑍𝑆) pode ser estimada também como sendo 1/2 𝐷𝑓𝑡. Já para a zona de diluição (𝐿𝑍𝐷), seu comprimento deve estar em torno de 3/2 de 𝐷𝑓𝑡

In [237]:
def comprimento_camara_combustao():
  L_cc = L_zp + L_zs + L_zd
  return L_cc

Determinação Preliminar da Distribuição de Ar
A zona secundária tem como função principal prover a completa combustão da mistura, logo, este é a intenção na especificação da quantidade de ar nesta zona. A situação que mais pode limitar a totalidade da combustão é a condição de operação mais rica. Esta condição é demarcada pela maior razão de equivalência global. Neste sentido, espera-se que a razão de equivalência global, até a segunda zona, seja ligeiramente pobre. Fica a critério do projetista determinar o valor exato, considerando as especificações do projeto. Neste trabalho, foi escolhido o valor de 0,8 como convencionado na literatura.

0,8 definido pelo Caio segunda referencias Pag 56

Quantidade relativa de ar até a zona secundária

In [238]:
def quant_ar_zona_secundaria(phi_global,rico,phi_zs):
  quanti_relativa_ar = (phi_global+rico)/phi_Zs
  return 0.8

Estimativa da porcentagem de ar destinada para o resfriamento das paredes do tubo de chama, com aprx de 10%  - página 58

In [239]:
def porcentagem_ar_resfriamento(m_ponto_arref, T3, m_ponto_3):
    m_ponto_arref = (0.1*T3 - 30) * m_ponto_3
    return m_ponto_arref

A vazão de ar na zona de diluição(𝑚̇𝑍d) é determinada através da expressão: - Página 58

In [240]:
def vazao_ar_zona_diluicao(m_ponto_zp, m_ponto_zs, m_ponto_arref, m_ponto_3):
    m_ponto_zd = (1 - (sum(m_ponto_zp, m_ponto_zs, m_ponto_arref))/m_ponto_3 )
    return m_ponto_3

### Projeto de arrefecimento do tubo de chama

Para que o motor opere em uma quantidade de temperatura que extenda a vida útil para a maioria das ligas metálicas é aconselhado que fique até 1100K. Já operando acima de 1300K corre risco de uma rápida oxidação - página 58

### Zona primária

Dentro dos limites da zona de recirculação, é considerado que todo seu
volume apresenta a mesma temperatura. Isto pode ser assumido devido à
forte recirculação dos gases nesta região, o que provoca uma distribuição de
temperatura mais homogênea. A temperatura estimada para esta zona é o
resultado da média ponderada entre a temperatura mais baixa, a dos gases de
entrada, 𝑇3
, e a temperatura mais alta, calculada pela seguinte equação: (equacao página59)

- Δ𝑇𝜙=1 o aumento de temperatura desenvolvido pela combustão
adiabática da mistura na situação de estequiometria. Este valor pode ser
obtido através de um programa de equilíbrio químico.


In [241]:
#Colocar os valores da variavel dentro do tanh para funcionar. Está dando errado que não consegue converter para float.
# variavel_dentro_tanh = 1.5475 * (10**-3) * (T3 + 108 * np.log(p_3) - 1863)
# eta_zr = 0.83 + (0.17 * mt.tanh(variavel_dentro_tanh))
def t_max_zr(T3, eta_zr, delta_T_phi):
    t_max_zr = T3 + eta_zr*delta_T_phi
    return t_max_zr

Para calcular a temperatura média nesta zona - página 60

In [242]:
def t_med_zr(T3, T_max_zr):
    T_med_zr = (1/3 * T3) + (2/3 * T_max_zr)
    return T_med_zr

Restante da zona primária  - página 60


In [243]:
#Colocar os valores da variavel dentro do tanh para funcionar. Está dando errado que não consegue converter para float.
# variavel_dentro_tanh = 1.5475 * (10**-3) * (T3 + 108 * np.log(p_3) - 1863)
# eta_zp = 0.92 + (0.12 * mt.tanh(variavel_dentro_tanh))
def t_saida_zp(T3, eta_zp, delta_T_zp):
    T_saida_zp = T3 + (eta_zp * delta_T_zp) 
    return T_saida_zp

Zona secundária
O mesmo método se aplica para o cálculo da temperatura na zona secundária,
considerando que a temperatura dos gases que entram nesta região é igual à
𝑇𝑠𝑎í𝑑𝑎,𝑍𝑃, e a temperatura final segue o modelo anterior, diferenciando-se apenas
na eficiência da combustão nesta zona
 - página 60

In [244]:
#se quiserem da para fazer uma funcao só pegue a temp da zona primaria e temp da zona secundaria
def t_saida_zp(T3, eta_zs, delta_T_zs):
    T_saida_zs = T3 + (eta_zs * delta_T_zs) 
    return T_saida_zs

Diferenca do eta_ns para misturas ricas e misturas pobres - página 60

In [245]:
#misturas ricas
eta_zs = 1/phi_Zs

#misturas pobres
# log log 1⁄𝜂𝑍𝑆 = 0,911log 𝜓𝑇3 + 8,02𝜙 − 1,097 + 𝐷∗ ------ tirar duvidas 

# sendo 𝐷
# ∗ um parâmetro da influência da perda de pressão total ao longo da
# câmara e 𝜓𝑇3 um parâmetro chamado de carga cinética do combustível [17],
# definidos abaixo



Definindo D* e psi_T3 - página 61

- 𝐷∗ um parâmetro da influência da perda de pressão total ao longo da
câmara 
- 𝜓𝑇3 um parâmetro chamado de carga cinética do combustível [17],
definidos abaixo:


- 𝑉𝑍𝑆 corresponde ao volume da zona secundária, que, para câmaras anulares sem
cúpula corresponde a

In [246]:
#definindo D_ast 
D_ast = 0.736 - 0.0173 * P3/delta_p

#Definindo V_zs
def V_zs(A_ft, L_sz):
    V_zs = A_ft * L_sz
    return V_zs


#Definindo psi_T3
def phi_t3(phi_Zs, m_ponto_comb, V_zs, P):
    psi_t3 = (10 ** (-3.054 * (phi_Zs **-1.25))) * (T3 ** (1.2327 * (phi_Zs ** -1.205))) * m_ponto_comb/ V_zs * (P**(2*phi_Zs))
    return psi_t3

### Zona de diluição
O método para determinação da temperatura na zona de diluição é o mesmo que o para
zona secundária, utilizando os termos referentes a zona de diluição e considerando que 𝜙𝑍𝐷 =
𝜙𝑔𝑙𝑜𝑏𝑎𝑙 e que 𝑉𝑍𝐷 = 𝐴𝑓𝑡 . 𝐿𝑍𝐷). Por fim, toma-se 𝑇𝑠𝑎í𝑑𝑎,𝑍𝐷 como a 𝑇4
. Portando esta temperatura, pode-se calcular o
perfil radial de temperatura e qualidade transversal de temperatura